In [1]:
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
# 데이터 불러오기
train_data = pd.read_csv("data/merge_train.csv")
test_data = pd.read_csv("data/merge_test.csv")

/tmp/ipykernel_24238/3068496492.py:3: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv("data/merge_test.csv")


In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306795 entries, 0 to 306794
Data columns (total 19 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   user_id                 306795 non-null  int64  
 1   isbn                    306795 non-null  object 
 2   rating                  306795 non-null  int64  
 3   Unnamed: 0              306795 non-null  int64  
 4   book_title              306795 non-null  object 
 5   book_author             306795 non-null  object 
 6   year_of_publication     306795 non-null  float64
 7   publisher               306795 non-null  object 
 8   img_url                 306795 non-null  object 
 9   language                306795 non-null  object 
 10  summary                 187709 non-null  object 
 11  img_path                306795 non-null  object 
 12  category                305901 non-null  object 
 13  summary_topic           187711 non-null  float64
 14  category_topic      

In [4]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76699 entries, 0 to 76698
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   user_id                 76699 non-null  int64  
 1   isbn                    76699 non-null  object 
 2   rating                  76699 non-null  int64  
 3   location                76699 non-null  object 
 4   age                     76699 non-null  float64
 5   Unnamed: 0              76699 non-null  int64  
 6   book_title              76699 non-null  object 
 7   book_author             76699 non-null  object 
 8   year_of_publication     76699 non-null  float64
 9   publisher               76699 non-null  object 
 10  img_url                 76699 non-null  object 
 11  language                76699 non-null  object 
 12  summary                 46907 non-null  object 
 13  img_path                76699 non-null  object 
 14  category                76491 non-null

In [5]:
test_data['year_of_publication'].isna().sum()

0

### age, year_of_publication 범주화

- 최종 모델에서는 범주화하지 않음
- 스킵

In [29]:
def age_map(x: int) -> int:
    x = int(x)
    if x < 20:
        return 1
    elif x >= 20 and x < 30:
        return 2
    elif x >= 30 and x < 40:
        return 3
    elif x >= 40 and x < 50:
        return 4
    elif x >= 50 and x < 60:
        return 5
    else:
        return 6

In [30]:
train_data['age'] = train_data['age'].apply(age_map)
test_data['age'] = test_data['age'].apply(age_map)

In [31]:
def year_map(x: int) -> int:
    x = int(x)
    if x < 1900:
        return 0
    elif x >= 1900 and x < 1910:
        return 1
    elif x >= 1910 and x < 1920:
        return 2
    elif x >= 1920 and x < 1930:
        return 3
    elif x >= 1930 and x < 1940:
        return 4
    elif x >= 1940 and x < 1950:
        return 5
    elif x >= 1950 and x < 1960:
        return 6
    elif x >= 1960 and x < 1970:
        return 7
    elif x >= 1970 and x < 1980:
        return 8
    elif x >= 1980 and x < 1990:
        return 9
    elif x >= 1990 and x < 2000:
        return 10
    else:
        return 11


In [32]:
train_data['year_of_publication'] = train_data['year_of_publication'].fillna(int(train_data['year_of_publication'].mean()))
train_data['year_of_publication'] = train_data['year_of_publication'].apply(year_map)
test_data['year_of_publication'] = test_data['year_of_publication'].fillna(int(test_data['year_of_publication'].mean()))
test_data['year_of_publication'] = test_data['year_of_publication'].apply(year_map)

In [125]:
# age와 year_of_publication 범주화 확인
train_data.nunique()

user_id                    59803
isbn                      129777
rating                        10
Unnamed: 0                129777
book_title                115473
book_author                52679
year_of_publication           92
publisher                    936
img_url                   129777
language                      24
summary                    69758
img_path                  129777
category                      15
summary_topic                350
category_topic               112
country_publisher_code     13690
location                   13888
age                           91
location_country             246
dtype: int64

In [126]:
test_data.nunique()

user_id                   26167
isbn                      52000
rating                        1
location                   8041
age                          86
Unnamed: 0                52000
book_title                46956
book_author               23060
year_of_publication          81
publisher                   845
img_url                   52000
language                     17
summary                   29632
img_path                  52000
category                     15
summary_topic               348
category_topic              112
country_publisher_code     9827
location_country            170
dtype: int64

## Feature 뽑기

In [18]:
# 원하는 피처만 뽑아내기 
train_data = train_data[['user_id', 'isbn', 'language', 'rating', 'location_country', 'age', 'country_publisher_code', 'summary_topic', 'category', 'publisher', 'year_of_publication', 'book_author']]
test_data = test_data[['user_id', 'isbn', 'language', 'rating', 'location_country', 'age', 'country_publisher_code', 'summary_topic', 'category', 'publisher', 'year_of_publication', 'book_author']]

In [19]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306795 entries, 0 to 306794
Data columns (total 12 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   user_id                 306795 non-null  object 
 1   isbn                    306795 non-null  object 
 2   language                306795 non-null  object 
 3   rating                  306795 non-null  int64  
 4   location_country        306795 non-null  object 
 5   age                     306795 non-null  float64
 6   country_publisher_code  306795 non-null  object 
 7   summary_topic           306795 non-null  object 
 8   category                306795 non-null  object 
 9   publisher               306795 non-null  object 
 10  year_of_publication     306795 non-null  float64
 11  book_author             306795 non-null  object 
dtypes: float64(2), int64(1), object(9)
memory usage: 28.1+ MB


In [20]:
train_data.fillna('NaN', inplace=True)

In [21]:
# train_data['age'] = train_data['age'].astype(str)
train_data['summary_topic'] = train_data['summary_topic'].astype(str)
# train_data['category_topic'] = train_data['category_topic'].astype(str)
train_data['user_id'] = train_data['user_id'].astype(str)
# train_data['year_of_publication'] = train_data['year_of_publication'].astype(str)
# train_data['year_of_publication'] = train_data['year_of_publication'].astype(str)
# train_data = train_data.apply(lambda x: x.astype(str))
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306795 entries, 0 to 306794
Data columns (total 12 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   user_id                 306795 non-null  object 
 1   isbn                    306795 non-null  object 
 2   language                306795 non-null  object 
 3   rating                  306795 non-null  int64  
 4   location_country        306795 non-null  object 
 5   age                     306795 non-null  float64
 6   country_publisher_code  306795 non-null  object 
 7   summary_topic           306795 non-null  object 
 8   category                306795 non-null  object 
 9   publisher               306795 non-null  object 
 10  year_of_publication     306795 non-null  float64
 11  book_author             306795 non-null  object 
dtypes: float64(2), int64(1), object(9)
memory usage: 28.1+ MB


In [22]:
test_data.fillna('NaN', inplace=True)
test_data['summary_topic'] = test_data['summary_topic'].astype(str)
# test_data['category_topic'] = test_data['category_topic'].astype(str)
test_data['user_id'] = test_data['user_id'].astype(str)
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76699 entries, 0 to 76698
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   user_id                 76699 non-null  object 
 1   isbn                    76699 non-null  object 
 2   language                76699 non-null  object 
 3   rating                  76699 non-null  int64  
 4   location_country        76699 non-null  object 
 5   age                     76699 non-null  float64
 6   country_publisher_code  76699 non-null  object 
 7   summary_topic           76699 non-null  object 
 8   category                76699 non-null  object 
 9   publisher               76699 non-null  object 
 10  year_of_publication     76699 non-null  float64
 11  book_author             76699 non-null  object 
dtypes: float64(2), int64(1), object(9)
memory usage: 7.0+ MB


In [23]:
X_train = train_data.drop(["isbn", "rating"], axis=1)
y_train = train_data["rating"]
X_test = test_data.drop(["isbn", "rating"], axis=1)
y_test = test_data["rating"]

In [24]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76699 entries, 0 to 76698
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   user_id                 76699 non-null  object 
 1   language                76699 non-null  object 
 2   location_country        76699 non-null  object 
 3   age                     76699 non-null  float64
 4   country_publisher_code  76699 non-null  object 
 5   summary_topic           76699 non-null  object 
 6   category                76699 non-null  object 
 7   publisher               76699 non-null  object 
 8   year_of_publication     76699 non-null  float64
 9   book_author             76699 non-null  object 
dtypes: float64(2), object(8)
memory usage: 5.9+ MB


In [25]:
feature_list = [0, 1, 2, 4, 5, 6, 7, 9]

In [ ]:
model = CatBoostRegressor(iterations=3000,
                          learning_rate=0.001,
                          one_hot_max_size=10,
                          early_stopping_rounds=30,
                          depth=10,
                          min_child_samples=6,
                          loss_function='RMSE',
                          verbose=True,
                          cat_features=feature_list)  # cat_features 지정

## 테스트

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=42)

In [29]:
model.fit(X_train, y_train, eval_set=[(X_val, y_val)], cat_features=feature_list)

0:	learn: 2.4328811	test: 2.4328267	best: 2.4328267 (0)	total: 135ms	remaining: 6m 46s
1:	learn: 2.4325587	test: 2.4324447	best: 2.4324447 (1)	total: 282ms	remaining: 7m 2s
2:	learn: 2.4322283	test: 2.4320528	best: 2.4320528 (2)	total: 390ms	remaining: 6m 29s
3:	learn: 2.4319135	test: 2.4316809	best: 2.4316809 (3)	total: 477ms	remaining: 5m 57s
4:	learn: 2.4315878	test: 2.4312912	best: 2.4312912 (4)	total: 559ms	remaining: 5m 34s
5:	learn: 2.4312667	test: 2.4309182	best: 2.4309182 (5)	total: 652ms	remaining: 5m 25s
6:	learn: 2.4309464	test: 2.4305402	best: 2.4305402 (6)	total: 716ms	remaining: 5m 5s
7:	learn: 2.4306232	test: 2.4301563	best: 2.4301563 (7)	total: 767ms	remaining: 4m 46s
8:	learn: 2.4303105	test: 2.4297852	best: 2.4297852 (8)	total: 797ms	remaining: 4m 25s
9:	learn: 2.4299916	test: 2.4294039	best: 2.4294039 (9)	total: 870ms	remaining: 4m 20s
10:	learn: 2.4296639	test: 2.4290125	best: 2.4290125 (10)	total: 964ms	remaining: 4m 22s
11:	learn: 2.4293464	test: 2.4286364	best: 

In [79]:
y_pred = model.predict(X_test)

In [80]:
result = pd.DataFrame({'user_id': test_data['user_id'],
                       'isbn': test_data['isbn'],
                       'rating': y_pred})

In [82]:
result.to_csv('CatBoost_i900_lr005_d10.csv', index=False)

In [34]:

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=42)

In [35]:
# model.fit(X_train, y_train)
model.fit(X_train, y_train, eval_set=[(X_val, y_val)], cat_features=feature_list, )

0:	learn: 2.4170107	test: 2.4139921	best: 2.4139921 (0)	total: 193ms	remaining: 9m 38s
1:	learn: 2.4024788	test: 2.3967767	best: 2.3967767 (1)	total: 359ms	remaining: 8m 58s
2:	learn: 2.3890548	test: 2.3807562	best: 2.3807562 (2)	total: 529ms	remaining: 8m 48s
3:	learn: 2.3769982	test: 2.3664079	best: 2.3664079 (3)	total: 605ms	remaining: 7m 33s
4:	learn: 2.3661237	test: 2.3533886	best: 2.3533886 (4)	total: 681ms	remaining: 6m 48s
5:	learn: 2.3562223	test: 2.3415189	best: 2.3415189 (5)	total: 787ms	remaining: 6m 32s
6:	learn: 2.3470506	test: 2.3305520	best: 2.3305520 (6)	total: 871ms	remaining: 6m 12s
7:	learn: 2.3386098	test: 2.3205098	best: 2.3205098 (7)	total: 1s	remaining: 6m 16s
8:	learn: 2.3312738	test: 2.3117314	best: 2.3117314 (8)	total: 1.13s	remaining: 6m 17s
9:	learn: 2.3244429	test: 2.3036757	best: 2.3036757 (9)	total: 1.2s	remaining: 6m
10:	learn: 2.3177505	test: 2.2957324	best: 2.2957324 (10)	total: 1.33s	remaining: 6m 2s
11:	learn: 2.3100689	test: 2.2864654	best: 2.28646

In [ ]:
# 성능평가
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("RMSE:", np.sqrt(mse))

## Hyper parameter tuning

Cross-validation + Grid Search
- 교차 검증을 사용한 Grdi search 방법
- train set, validation set 을 한번만 나누지 않고 k번 사용
- sklearn.model_selection 의 GridSearchCV 사용

In [78]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [79]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=42)

In [88]:
param_grid = {
    'learning_rate': [0.04, 0.02, 0.01, 0.005, 0.001],
    'depth': [5, 6, 7, 8, 9, 10],
    'l2_leaf_reg': [1, 2, 3, 4, 5],
    'min_child_samples': [3, 5, 7, 16]
    # 'iterations': 5000,
    # ,'eval_metric': 'RMSE',
}

In [97]:
from sklearn.model_selection import KFold

model = CatBoostRegressor()
cv = KFold(n_splits=5, shuffle=True, random_state=42)

In [100]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(model, param_grid, cv=cv, verbose=2)

In [ ]:
grid_search.fit(X_train, y_train, cat_features=feature_list)

In [ ]:
# 결과 확인
print(grid_search.best_params_)
print(grid_search.best_score_)